# Plotting for results

This notebook produces all results plots.
It generates some gap in the data, fill with a method (filter, MDS ...), compute metrics and then makes all relevant plots

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from fastcore.test import *
from fastcore.basics import *
from meteo_imp.utils import *
from meteo_imp.gaussian import *
from meteo_imp.data_preparation import MeteoDataTest
from meteo_imp.kalman.filter import *
from meteo_imp.kalman.filter import get_test_data
from meteo_imp.data import *
from meteo_imp.kalman.fastai import *
from fastcore.transform import *
from fastai.learner import *

import pykalman
from typing import *

import numpy as np
import pandas as pd
import torch
from torch import Tensor
from torch.distributions import MultivariateNormal

from timeit import timeit
import polars as pl
import altair as alt

from tqdm.auto import tqdm

In [ ]:
reset_seed()

In [ ]:
hai = pd.read_parquet(hai_path64)
hai_era = pd.read_parquet(hai_era_path64)

In [ ]:
pipe, block_ids = imp_pipeline(hai, hai_era, ['TA'], gap_len=10, block_len=100, control_lags = [1], n_rep=1)
pipe = Pipeline(pipe[:-3])

In [ ]:
item = pipe([401,10])

## Kalman Filter

take a `MeteoImp Df` and gap fill it using the given model

In [ ]:
model = torch.load("trained_30_jan_v1.pickle")

In [ ]:
dls = imp_dataloader(hai, hai_era, var_sel = ['TA', 'SW_IN', 'VPD'], block_len=300, gap_len=10, bs=20, control_lags=[1], n_rep=3).cpu()

In [ ]:
learn = Learner(dls, model, KalmanLoss(only_gap=True), cbs = [Float64Callback], metrics=imp_metrics)

In [ ]:
model.H.dtype

torch.float64

In [ ]:
hai.dtypes

TA       float64
SW_IN    float64
VPD      float64
dtype: object

In [ ]:
pred, targ, losses, metrics = get_results(learn, items = [item])
pred[0].mean[targ[0].mask] = targ[0].data[targ[0].mask]

In [ ]:
class KalmanImputation:
    def __init__(self, learn): store_attr()
    def __call__(self, item):
        pred, targ, losses, metrics = get_results(self.learn, items = [item])
        pred[0].mean[targ[0].mask] = targ[0].data[targ[0].mask]
        return pred[0].mean

In [ ]:
k_imp = KalmanImputation(learn)

In [ ]:
display_as_row({'pred': k_imp(item)[45:55],'data': item.data[45:55], 'mask': item.mask[45:55]}, hide_idx=False)

,TA,SW_IN,VPD
time,,,
2002-04-16 14:30:00,5.8200,240.8500,1.9980
2002-04-16 15:00:00,5.8800,271.7700,2.0970
2002-04-16 15:30:00,5.9000,188.3400,2.0410
2002-04-16 16:00:00,5.8200,114.6600,2.0300
2002-04-16 16:30:00,6.0300,153.7900,2.1810
2002-04-16 17:00:00,5.8231,100.5100,2.0130
2002-04-16 17:30:00,5.6213,89.0400,1.9860
2002-04-16 18:00:00,5.4872,24.1600,1.7470
2002-04-16 18:30:00,5.3349,22.1800,1.4840


## MDS

Need to call R from python

In [ ]:
import rpy2

In [ ]:
rpy2.__version__

'3.5.7'

In [ ]:
import rpy2.rinterface

In [ ]:
from rpy2.robjects.packages import importr

In [ ]:
import rpy2.robjects
from rpy2.robjects import r
import rpy2.robjects as ro
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri

In [ ]:
r.print('here::here("R/REddyProc_tools.R")')

[1] "here::here(\"R/REddyProc_tools.R\")"


"'here::here(""R/REddyProc_tools.R"")'"


In [ ]:
from pyprojroot import here

In [ ]:
path_r = str(here("R/REddyProc_tools.R"))

In [ ]:
r.source(path_r);

In [ ]:
%load_ext rpy2.ipython

In [ ]:
def importr_install(pkg):
    try:
        importr(pkg)
    except:
        utils = importr('utils')
        utils.chooseCRANmirror(ind=1)
        utils.install_packages(pkg)
        importr(pkg) 

In [ ]:
def setupR():
    importr_install('tidyverse')
    importr_install('REddyProc')
    importr('lubridate')
    path_r = str(here("R/REddyProc_tools.R"))
    r.source(path_r) # R functions
    r("""toR_timestamp <- function(x){
   x$TIMESTAMP_END = as_datetime(x$TIMESTAMP_END) 
    x
     }""")

In [ ]:
def pd2R(x):
    with ro.default_converter + pandas2ri.converter:
        return ro.conversion.py2rpy(x) 

In [ ]:
def R2pd(x):
    with ro.default_converter + pandas2ri.converter:
        return ro.conversion.rpy2py(x) 

In [ ]:
setupR()

### Experiment

The time series needs to be at least 90 days long, we we add 45 days before and after the gap

There is a problem with conversions of timestamps between R and Python, so convert to string in Python and then back to datetime in R

In [ ]:
def item2REddy(item, var, df):
    " Add context around item for supporting REddyProc"
    start = np.argwhere(hai.index == item.data.index[0]).squeeze()
    end = np.argwhere(hai.index == item.data.index[-1]).squeeze()
    start = start - 45 * 24 * 2 # 90 days
    end = end + 45 * 24 * 2
    REddy_df = hai.iloc[start:end].assign(gap = (~item.mask[var]).astype(int)).fillna({'gap': 0})
    return REddy_df.reset_index().astype({'time': str}).rename(columns={'time': 'TIMESTAMP_END'})

In [ ]:
REddy_df = item2REddy(item, 'TA', hai)

REddy_df_r = r.toR_timestamp(pd2R(REddy_df))

filled = R2pd(r.fill_gaps_EProc(REddy_df_r, "TA"))

In [ ]:
filled

,TA_orig,TA_f,TA_fqc,TA_fall,TA_fall_qc,TA_fnum,TA_fsd,TA_fmeth,TA_fwin
1,5.35,5.35,0.0,5.35,NaN,NaN,NaN,NaN,NaN
2,4.87,4.87,0.0,4.87,NaN,NaN,NaN,NaN,NaN
3,3.74,3.74,0.0,3.74,NaN,NaN,NaN,NaN,NaN
4,2.14,2.14,0.0,2.14,NaN,NaN,NaN,NaN,NaN
5,1.82,1.82,0.0,1.82,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
4415,14.89,14.89,0.0,14.89,NaN,NaN,NaN,NaN,NaN
4416,15.05,15.05,0.0,15.05,NaN,NaN,NaN,NaN,NaN
4417,15.06,15.06,0.0,15.06,NaN,NaN,NaN,NaN,NaN
4418,15.24,15.24,0.0,15.24,NaN,NaN,NaN,NaN,NaN


In [ ]:
REddy_df.set_index("TIMESTAMP_END") 

,TA,SW_IN,VPD,gap
TIMESTAMP_END,,,,
2002-03-01 16:00:00,5.35,102.16,3.755,0.0
2002-03-01 16:30:00,4.87,50.54,3.183,0.0
2002-03-01 17:00:00,3.74,25.75,1.957,0.0
2002-03-01 17:30:00,2.14,25.11,0.979,0.0
2002-03-01 18:00:00,1.82,5.03,1.066,0.0
...,...,...,...,...
2002-06-01 15:00:00,14.89,740.41,9.169,0.0
2002-06-01 15:30:00,15.05,679.92,10.075,0.0
2002-06-01 16:00:00,15.06,613.06,10.316,0.0


In [ ]:
filled = filled.set_index(pd.to_datetime(REddy_df.TIMESTAMP_END))

filled_item = filled.loc[item.data.index]

pred = item.data.copy()
pred.loc[~item.mask["TA"], "TA"] = filled_item["TA_f"][~item.mask["TA"]]

In [ ]:
pred[45:55], item.data[45:55]

(                           TA   SW_IN    VPD
 time                                        
 2002-04-16 14:30:00  5.820000  240.85  1.998
 2002-04-16 15:00:00  5.880000  271.77  2.097
 2002-04-16 15:30:00  5.900000  188.34  2.041
 2002-04-16 16:00:00  5.820000  114.66  2.030
 2002-04-16 16:30:00  6.030000  153.79  2.181
 2002-04-16 17:00:00  5.075889  100.51  2.013
 2002-04-16 17:30:00  5.185888   89.04  1.986
 2002-04-16 18:00:00  5.353310   24.16  1.747
 2002-04-16 18:30:00  5.384187   22.18  1.484
 2002-04-16 19:00:00  5.442061    8.29  1.440,
                        TA   SW_IN    VPD
 time                                    
 2002-04-16 14:30:00  5.82  240.85  1.998
 2002-04-16 15:00:00  5.88  271.77  2.097
 2002-04-16 15:30:00  5.90  188.34  2.041
 2002-04-16 16:00:00  5.82  114.66  2.030
 2002-04-16 16:30:00  6.03  153.79  2.181
 2002-04-16 17:00:00  5.77  100.51  2.013
 2002-04-16 17:30:00  5.82   89.04  1.986
 2002-04-16 18:00:00  5.45   24.16  1.747
 2002-04-16 18:30:00  5.09 

In [ ]:
def gap_fill_item(item, REddy_df, var, filled):
    
    filled = filled.set_index(pd.to_datetime(REddy_df.TIMESTAMP_END))
    filled_item = filled.loc[item.data.index]

    pred = item.data.copy()
    pred.loc[~item.mask[var], var] = filled_item[f"{var}_f"][~item.mask[var]]
    return pred

In [ ]:
gap_fill_item(item, REddy_df, "TA", filled)


,TA,SW_IN,VPD
time,,,
2002-04-15 16:00:00,5.05,147.99,0.663
2002-04-15 16:30:00,4.96,132.57,0.641
2002-04-15 17:00:00,4.89,72.85,0.623
2002-04-15 17:30:00,4.80,70.62,0.614
2002-04-15 18:00:00,4.70,25.56,0.564
...,...,...,...
2002-04-17 15:30:00,6.28,176.61,2.260
2002-04-17 16:00:00,6.19,162.65,2.069
2002-04-17 16:30:00,6.11,157.49,1.737


### MDSImputation

In [ ]:
class MDSImputation:
    def __init__(self, var, df): store_attr()
    def __call__(self, item):
        REddy_df = item2REddy(item, self.var, self.df)
        REddy_df_r = r.toR_timestamp(pd2R(REddy_df))
        filled = R2pd(r.fill_gaps_EProc(REddy_df_r, self.var))
        return gap_fill_item(item, REddy_df, self.var, filled)

In [ ]:
mds_imp = MDSImputation('TA', hai)

In [ ]:
mds_imp(item)

R[write to console]: New sEddyProc class for site 'ID'

R[write to console]: Initialized variable 'TA' with 10 real gaps for gap filling.

R[write to console]: Limited MDS algorithm for gap filling of 'TA.gap_0' with LUT(SW_IN only) and MDC.

R[write to console]: Look up table with window size of 7 days with SW_IN

R[write to console]: 10

R[write to console]: Finished gap filling of 'TA' in 0 seconds. Artificial gaps filled: 4419, real gaps filled: 10, unfilled (long) gaps: 0.



,TA,SW_IN,VPD
time,,,
2002-04-15 16:00:00,5.05,147.99,0.663
2002-04-15 16:30:00,4.96,132.57,0.641
2002-04-15 17:00:00,4.89,72.85,0.623
2002-04-15 17:30:00,4.80,70.62,0.614
2002-04-15 18:00:00,4.70,25.56,0.564
...,...,...,...
2002-04-17 15:30:00,6.28,176.61,2.260
2002-04-17 16:00:00,6.19,162.65,2.069
2002-04-17 16:30:00,6.11,157.49,1.737


## ERA Imputation

In [ ]:
class ERAImputation:
    def __call__(self, item):
        pred = item.control.copy()
        pred = pred.filter(regex=(".*[^_][^\d]$"))
        return pred.rename(columns=lambda x: x.replace("_ERA", ""))

In [ ]:
era_imp = ERAImputation()

In [ ]:
era_imp(item)

,TA,SW_IN,VPD
time,,,
2002-04-15 16:00:00,6.901,142.510,2.768
2002-04-15 16:30:00,6.963,42.318,2.865
2002-04-15 17:00:00,6.759,35.518,2.646
2002-04-15 17:30:00,6.555,28.323,2.428
2002-04-15 18:00:00,6.351,20.856,2.209
...,...,...,...
2002-04-17 15:30:00,7.845,272.214,2.309
2002-04-17 16:00:00,8.030,244.161,2.489
2002-04-17 16:30:00,8.216,139.238,2.668


## Metric

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
#| export
class MaskedMetric:
    def __init__(self, metric): store_attr()
    def __call__(self, item, pred):
        row_sel, col_sel = ~item.mask.all(1), ~item.mask.all(0)
        data, mask, pred = item.data.to_numpy(), item.mask.to_numpy(), pred.to_numpy()
        assert not mask[:, col_sel][row_sel, :].any() # gap is a rectangle
        return self.metric(data[:, col_sel][row_sel, :], pred[:, col_sel][row_sel, :])   

In [ ]:
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred, multioutput='raw_values'))

In [ ]:
MaskedMetric(nrmse)(item, pred)

array([0.16843353])

## Filling Comparison

Take a variable to be filled, makes an artificial gap of given len, tries to fill with 3 methods and return metrics for each of them, repeat `n_rep` times

In [ ]:
import random

In [ ]:
random.choices?

Signature: random.choices(population, weights=None, *, cum_weights=None, k=1)
Docstring:
Return a k sized list of population elements chosen with replacement.

If the relative weights or cumulative weights are not specified,
the selections are made with equal probability.
File:      ~/anaconda3/envs/data-science/lib/python3.10/random.py
Type:      method


In [ ]:
def fill_comparison(df, gap_len, var, n_rep):
    pipe, block_ids = imp_pipeline(hai, hai_era, [var], gap_len=gap_len, block_len=100, control_lags = [1], n_rep=1)
    pipe = Pipeline(pipe[:-3]) 
    items = pipe(random.choices(block_ids, k=n_rep))
    era_imp = ERAImputation()
    for item in items:
        

In [ ]:
start = start - 45 * 24 * 2
end = end + 45 * 24 * 2

In [ ]:
REddy_df = hai.iloc[start:end].assign(gap = (~item.mask['TA']).astype(int)).fillna({'gap': 0})

In [ ]:
REddy_df.loc[item.data.index][45:55]

,TA,SW_IN,VPD,gap
time,,,,
2002-04-16 14:30:00,5.82,240.85,1.998,0.0
2002-04-16 15:00:00,5.88,271.77,2.097,0.0
2002-04-16 15:30:00,5.90,188.34,2.041,0.0
2002-04-16 16:00:00,5.82,114.66,2.030,0.0
2002-04-16 16:30:00,6.03,153.79,2.181,0.0
2002-04-16 17:00:00,5.77,100.51,2.013,1.0
2002-04-16 17:30:00,5.82,89.04,1.986,1.0
2002-04-16 18:00:00,5.45,24.16,1.747,1.0
2002-04-16 18:30:00,5.09,22.18,1.484,1.0


In [ ]:
REddy_df = REddy_df.reset_index().astype({'time': str}).rename(columns={'time': 'TIMESTAMP_END'})

In [ ]:
REddy_df.dtypes

TIMESTAMP_END     object
TA               float64
SW_IN            float64
VPD              float64
gap              float64
dtype: object

In [ ]:
importr('lubridate')

rpy2.robjects.packages.Package as a <module 'lubridate'>

In [ ]:
%%R
toR_timestamp <- function(x){
   x$TIMESTAMP_END = as_datetime(x$TIMESTAMP_END) 
    x
}

In [ ]:
with ro.default_converter + pandas2ri.converter:
      REddy_df_r = ro.conversion.py2rpy(REddy_df)

In [ ]:
with ro.default_converter + pandas2ri.converter:
      pd_filled = ro.conversion.get_conversion().rpy2py(filled)

In [ ]:
REddy_df[REddy_df.gap !=0]

,TIMESTAMP_END,TA,SW_IN,VPD,gap
2210,2002-04-16 17:00:00,5.77,100.51,2.013,1.0
2211,2002-04-16 17:30:00,5.82,89.04,1.986,1.0
2212,2002-04-16 18:00:00,5.45,24.16,1.747,1.0
2213,2002-04-16 18:30:00,5.09,22.18,1.484,1.0
2214,2002-04-16 19:00:00,5.00,8.29,1.440,1.0
2215,2002-04-16 19:30:00,4.82,0.89,1.342,1.0
2216,2002-04-16 20:00:00,4.49,0.00,1.111,1.0
2217,2002-04-16 20:30:00,4.48,0.00,1.102,1.0
2218,2002-04-16 21:00:00,4.29,0.00,1.044,1.0
2219,2002-04-16 21:30:00,4.26,0.00,1.116,1.0


In [ ]:
pd_filled[pd_filled.TA_fqc!=0]

,TA_orig,TA_f,TA_fqc,TA_fall,TA_fall_qc,TA_fnum,TA_fsd,TA_fmeth,TA_fwin
2211,NaN,5.075889,1.0,5.075889,1.0,90.0,2.690751,2.0,14.0
2212,NaN,5.185888,1.0,5.185888,1.0,107.0,2.761757,2.0,14.0
2213,NaN,5.353310,1.0,5.353310,1.0,420.0,2.803028,2.0,14.0
2214,NaN,5.384187,1.0,5.384187,1.0,418.0,2.826861,2.0,14.0
2215,NaN,5.442061,1.0,5.442061,1.0,393.0,2.859437,2.0,14.0
2216,NaN,5.484188,1.0,5.484188,1.0,382.0,2.869645,2.0,14.0
2217,NaN,5.497816,1.0,5.497816,1.0,380.0,2.885646,2.0,14.0
2218,NaN,5.516737,1.0,5.516737,1.0,380.0,2.904111,2.0,14.0
2219,NaN,5.536526,1.0,5.536526,1.0,380.0,2.920621,2.0,14.0
2220,NaN,5.557000,1.0,5.557000,1.0,380.0,2.935087,2.0,14.0


In [ ]:
r("REddy_df$TIMESTAMP_END = as_datetime(REddy_df$TIMESTAMP_END)")
filled = r("fill_gaps_EProc(REddy_df, 'TA')")

R[write to console]: Error in h(simpleError(msg, call)) : 
  error in evaluating the argument 'x' in selecting a method for function 'as_datetime': object 'REddy_df' not found



RRuntimeError: Error in h(simpleError(msg, call)) : 
  error in evaluating the argument 'x' in selecting a method for function 'as_datetime': object 'REddy_df' not found


In [ ]:
%%R -i REddy_df
# print(slice(REddy_df, 1410:1420))
REddy_df$TIMESTAMP_END = as_datetime(REddy_df$TIMESTAMP_END)
class(REddy_df$TIMESTAMP_END)
str(REddy_df)
# REddy_df_r$TIMESTAMP_END[1412] - REddy_df_r$TIMESTAMP_END[1413]
# ((REddy_df$TIMESTAMP_END - lag(REddy_df$TIMESTAMP_END)))[1410:1420]
# which((REddy_df_r$TIMESTAMP_END - lag(REddy_df_r$TIMESTAMP_END)) != 30)
fill = fill_gaps_EProc(REddy_df, "TA")


In [ ]:
REddy_df.iloc[1410:1420]

In [ ]:
REddy_df_r[1]

-0.600000,-0.650000,-0.580000,...,4.320000,4.020000,3.990000


In [ ]:
REddy_df_r

TIMESTAMP_END,TA,SW_IN,VPD,gap
...,...,...,...,...


In [ ]:
r.[(REddy_df_r, "4080:4085")

Error:
! Problem while computing indices.
Caused by error:
! Invalid result of type <character>.
ℹ Indices must be positive or negative integers.
Run `rlang::last_error()` to see where the error occurred.


RRuntimeError: Error in (function (.data, ..., .preserve = FALSE)  : 
Caused by error:
! Invalid result of type <character>.
ℹ Indices must be positive or negative integers.


In [ ]:
r.fill_gaps_EProc(REddy_df_r, "TA")

R[write to console]: Error in fCheckHHTimeSeries(Data[[ColPOSIXTime]], DTS = DTS, "sEddyProc.initialize") : 
  sEddyProc.initialize:::fCheckHHTimeSeries::: Time stamp is not equidistant (half-)hours in rows: 4083, 14501, 21555, 31973, 39363, 49445, 56835, 66917, 74307, 84725, 91779, 102197, 109251, 119669, 126723, 137141, 144531, 154613, 162003, 172085, 179475, 189893, 196947, 207365, 214419, 224837



RRuntimeError: Error in fCheckHHTimeSeries(Data[[ColPOSIXTime]], DTS = DTS, "sEddyProc.initialize") : 
  sEddyProc.initialize:::fCheckHHTimeSeries::: Time stamp is not equidistant (half-)hours in rows: 4083, 14501, 21555, 31973, 39363, 49445, 56835, 66917, 74307, 84725, 91779, 102197, 109251, 119669, 126723, 137141, 144531, 154613, 162003, 172085, 179475, 189893, 196947, 207365, 214419, 224837


In [ ]:
REddy_df[45:55]

,TIMESTAMP_END,TA,SW_IN,VPD,gap
45,2002-04-16 14:30:00,5.82,240.85,1.998,0
46,2002-04-16 15:00:00,5.88,271.77,2.097,0
47,2002-04-16 15:30:00,5.90,188.34,2.041,0
48,2002-04-16 16:00:00,5.82,114.66,2.030,0
49,2002-04-16 16:30:00,6.03,153.79,2.181,0
50,2002-04-16 17:00:00,5.77,100.51,2.013,1
51,2002-04-16 17:30:00,5.82,89.04,1.986,1
52,2002-04-16 18:00:00,5.45,24.16,1.747,1
53,2002-04-16 18:30:00,5.09,22.18,1.484,1
54,2002-04-16 19:00:00,5.00,8.29,1.440,1


In [ ]:
r_from_pd_df[0]

2002-04-1...,...,2002-04-1...,2002-04-1...,2002-04-1...


In [ ]:
r.remove_var_gap()

R[write to console]: Error in data[as.logical(data$gap), remove_vars] <- NA : 
  argument "data" is missing, with no default



RRuntimeError: Error in data[as.logical(data$gap), remove_vars] <- NA : 
  argument "data" is missing, with no default


In [ ]:
from functools import partial
from rpy2.ipython import html
html.html_rdataframe=partial(html.html_rdataframe, table_class="docutils")

## Generate gaps in data